In [2]:
import d2d
import importlib
importlib.reload(d2d)
import numpy as np
import tensorflow as tf
import pandas as pd

In [3]:
linear, lstm_model, dnn_model, df_all_chan,input_columns  = d2d.import_data()

da = df_all_chan.to_numpy()

chan_mean = da[:,0:2495].flatten().mean()
chan_std = da[:,0:2495].flatten().std()
discharge_mean = da[:,2496].flatten().mean()
discharge_std = da[:,2496].flatten().std()

da[:,0:2495] = (da[:,0:2495] - chan_mean) / chan_std
da[:,2496] = (da[:,2496] - discharge_mean) / discharge_std
df_all_chan

Keys: <KeysViewHDF5 ['DAS Data', 'Discharge', 'Times']>


,0,1,2,3,4,5,6,7,8,9,...,2487,2488,2489,2490,2491,2492,2493,2494,2495,Discharge
0,0.055370,0.132891,0.075625,0.838384,0.092913,0.067835,0.123561,0.831771,0.081936,0.333693,...,0.797420,0.186061,0.245533,0.112796,0.951247,-0.005196,0.075232,0.105298,4160.545583,-0.199862
1,2.609165,2.642731,2.592007,-0.425048,-0.448878,-0.351154,-0.345852,-0.572293,-0.602395,-0.588788,...,2.639816,2.626478,2.622412,2.635967,2.628929,2.636206,2.631859,2.628562,8178.032001,-0.203829
2,2.641601,2.626772,2.582383,-0.432204,-0.455639,-0.353222,-0.353055,-0.574172,-0.602123,-0.589121,...,2.618368,2.633073,2.625496,2.622513,2.623606,2.620045,2.620419,2.634475,8163.918322,-0.205529
3,2.608541,2.623200,2.590291,-0.440592,-0.460897,-0.360812,-0.356249,-0.572446,-0.602283,-0.590442,...,2.614139,2.607113,2.605096,2.638779,2.602809,2.621506,2.612316,2.596673,8161.652382,-0.207229
4,2.623933,2.609213,2.592907,-0.427993,-0.458665,-0.361094,-0.358925,-0.571576,-0.601865,-0.591103,...,2.624064,2.638370,2.632476,2.601060,2.621953,2.607497,2.641087,2.636136,8234.212679,-0.208929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62740,2.644459,2.640414,2.592968,-0.353144,-0.426673,-0.302467,-0.440626,-0.605681,-0.595740,-0.598741,...,2.639635,2.641696,2.649847,2.639092,2.654711,2.607855,2.646024,2.648450,8189.577468,-0.416159
62741,2.666290,2.630778,2.594296,-0.351444,-0.427582,-0.286583,-0.434774,-0.606149,-0.596309,-0.597133,...,2.642260,2.632740,2.637970,2.625693,2.653305,2.628002,2.618845,2.624307,8194.036074,-0.416357
62742,2.637746,2.629654,2.595306,-0.352068,-0.428374,-0.280593,-0.438196,-0.606441,-0.596674,-0.595969,...,2.662767,2.637987,2.630461,2.613858,2.636926,2.617872,2.613045,2.623067,8220.779280,-0.416555
62743,2.636491,2.639474,2.620006,-0.354822,-0.427148,-0.275793,-0.433923,-0.606532,-0.596980,-0.595063,...,2.637306,2.630241,2.654636,2.651023,2.606471,2.627836,2.644935,2.651291,8207.721162,-0.416753


In [16]:
test_data_frame = pd.DataFrame({'0':range(0,10000), 'Discharge':range(0,10000)})
input_columns = ['0']

In [6]:
print('Number of DAS observations: %d'%df_all_chan.shape[0])
window_input_width = 200
print('Number of tf Dataset windows: %d'%np.floor(df_all_chan.shape[0]/window_input_width))

Number of DAS observations: 62745
Number of tf Dataset windows: 313


In [17]:
multi_step_window = d2d.WindowGenerator(test_data_frame,
    input_width=window_input_width, label_width=1, shift=0,
    label_columns=['Discharge'],
    input_columns=input_columns,
    shuffle=False)

multi_step_window_shuffled = d2d.WindowGenerator(test_data_frame,
    input_width=window_input_width, label_width=1, shift=0,
    label_columns=['Discharge'],
    input_columns=input_columns,
    shuffle=True)

In [18]:
ntest = tf.data.experimental.cardinality(multi_step_window.test)
nval = multi_step_window.val.cardinality().numpy()
ntrain = multi_step_window.train.cardinality().numpy()
print('Size of test ds: %d'%ntest)
print('Size of val ds: %d'%nval)
print('Size of train ds: %d'%ntrain)
print('Sum of ds sizes = %d'%(ntest+nval+ntrain))

Size of test ds: 16
Size of val ds: 15
Size of train ds: 123
Sum of ds sizes = 154


In [19]:
multi_step_window.example

(<tf.Tensor: shape=(64, 200, 1), dtype=float32, numpy=
 array([[[  0.],
         [  1.],
         [  2.],
         ...,
         [197.],
         [198.],
         [199.]],
 
        [[  1.],
         [  2.],
         [  3.],
         ...,
         [198.],
         [199.],
         [200.]],
 
        [[  2.],
         [  3.],
         [  4.],
         ...,
         [199.],
         [200.],
         [201.]],
 
        ...,
 
        [[ 61.],
         [ 62.],
         [ 63.],
         ...,
         [258.],
         [259.],
         [260.]],
 
        [[ 62.],
         [ 63.],
         [ 64.],
         ...,
         [259.],
         [260.],
         [261.]],
 
        [[ 63.],
         [ 64.],
         [ 65.],
         ...,
         [260.],
         [261.],
         [262.]]], dtype=float32)>,
 <tf.Tensor: shape=(64, 1, 1), dtype=float32, numpy=
 array([[[199.]],
 
        [[200.]],
 
        [[201.]],
 
        [[202.]],
 
        [[203.]],
 
        [[204.]],
 
        [[205.]],
 
       

In [20]:
multi_step_window_shuffled.example

(<tf.Tensor: shape=(64, 200, 1), dtype=float32, numpy=
 array([[[5533.],
         [5534.],
         [5535.],
         ...,
         [5730.],
         [5731.],
         [5732.]],
 
        [[7509.],
         [7510.],
         [7511.],
         ...,
         [7706.],
         [7707.],
         [7708.]],
 
        [[6808.],
         [6809.],
         [6810.],
         ...,
         [7005.],
         [7006.],
         [7007.]],
 
        ...,
 
        [[8099.],
         [8100.],
         [8101.],
         ...,
         [8296.],
         [8297.],
         [8298.]],
 
        [[2213.],
         [2214.],
         [2215.],
         ...,
         [2410.],
         [2411.],
         [2412.]],
 
        [[1667.],
         [1668.],
         [1669.],
         ...,
         [1864.],
         [1865.],
         [1866.]]], dtype=float32)>,
 <tf.Tensor: shape=(64, 1, 1), dtype=float32, numpy=
 array([[[5732.]],
 
        [[7708.]],
 
        [[7007.]],
 
        [[8143.]],
 
        [[2587.]],
 
      

In [ ]:
history = d2d.compile_and_fit(linear, multi_step_window, learning_rate = 0.001)

In [ ]:
val_performance = this_model.evaluate(multi_step_window.val)
performance = this_model.evaluate(multi_step_window.test, verbose=0)
